In [1]:
import pandas as pd
import urllib

import numpy as np

import json

from tqdm.autonotebook import tqdm

#%matplotlib inline

tqdm.pandas()

import dask.dataframe as dd

from dask.multiprocessing import get
from dask.diagnostics import ProgressBar

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.display import display


/tmp/ipykernel_177493/3514129619.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import urllib3

In [3]:
http = urllib3.PoolManager()

In [5]:
from config_batch import * 

# Functions

In [6]:
ws_hostname = "127.0.1.1"
ws_hostname = "172.27.0.64"

# ws_hostname = "192.168.1.3"


In [7]:
def call_ws(addr_data, check_result=True, structured_osm=False): #lg = "en,fr,nl"
    t = datetime.now()
    
    params = urllib.parse.urlencode({"street": addr_data[street_field],
                                     "housenumber": addr_data[housenbr_field],
                                     "city": addr_data[city_field],
                                     "postcode": addr_data[postcode_field],
                                     "country": addr_data[country_field],
                                     "check_result" : "yes" if check_result else "no",
                                     "struct_osm" : "yes" if structured_osm else "no"
                                    })
    url = f"http://{ws_hostname}:5000/search/?{params}"
    
#     print(url)
    try:
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             print(res)
            res["time"] = (datetime.now() - t).total_seconds()
            return res
    except Exception as e:
        return str(e)
    

In [9]:
def call_ws_batch(addr_data, mode="geo", with_reject=False, check_result=True, structured_osm=False): #lg = "en,fr,nl"
#     print(addr_data)
#     print(addr_data.shape)
#     print()
    file_data = addr_data.rename(columns = {
        street_field : "street",
        housenbr_field: "housenumber",
        postcode_field: "postcode",
        city_field: "city",
        country_field: "country",
        addr_key_field : "addr_key"
    }).to_csv(index=False)
    
    r = http.request(
    'POST',
    f'http://{ws_hostname}:5000/batch',
    fields= { 
        'media': ('addresses.csv', file_data),
        'mode': mode,
        "with_rejected" : "yes" if with_reject else "no",
        "check_result" : "yes" if check_result else "no",
        "struct_osm" : "yes" if structured_osm else "no",
        #"extra_house_nbr": "no"
    })
    
    try:
        res = pd.DataFrame(json.loads(r.data.decode('utf-8')))
    except ValueError:
        print("Cannot decode result:")
        print(r.data.decode('utf-8'))
        print(json.loads(r.data.decode('utf-8')))
        return 
#     display(res)
    return res

In [62]:
def expand_json(addresses):
    addresses["status"]= addresses.json.apply(lambda d: "error" if "error" in d else "match" if "match" in d else "rejected")
    addresses["time"]  = addresses.json.apply(lambda d: d["time"])

    addresses["timing"]  = addresses.json.apply(lambda d: d["timing"] if "timing" in d else {})

    addresses["method"]= addresses.json.apply(lambda d: d["match"][0]["method"] if len(d)>0 and "match" in d else "none")
    
    for field in ["street", "number", "postcode", "city"]:
        addresses["addr_out_"+field]= addresses.json.apply(lambda d: d["match"][0]["addr_out_"+field] if len(d)>0 and "match" in d else "")
    return 

# Calls

## Single address calls

In [337]:
res=call_ws({street_field:   "Av. Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgium"}, check_result=False, structured_osm=False)
res

{'match': [{'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '1060',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'display_name': '20, Avenue Fonsny - Fonsnylaan, Saint-Gilles - Sint-Gillis, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1060, België / Belgique / Belgien',
   'in_house_nbr': '20',
   'lat': '50.8358677',
   'lon': '4.3385087',
   'lpost_house_nbr': '20',
   'lpost_unit': '',
   'method': 'fast',
   'osm_addr_in': 'Av. Fonsny, 20, 1060 Saint-Gilles, Belgium',
   'place_id': 182128,
   'place_rank': 30}],
 'reject': [{'addr_out_city': 'Saint-Gilles - Sint-Gillis',
   'addr_out_country': 'België / Belgique / Belgien',
   'addr_out_number': '20',
   'addr_out_postcode': '1060',
   'addr_out_street': 'Avenue Fonsny - Fonsnylaan',
   'display_name': 'DAE (Smals), 20, Avenue Fonsny - Fonsnylaan, Saint-Gilles 

## Batch calls (row by row)

In [363]:
# KBO dataset
addresses = get_addresses("address.csv.gz")
# addresses = addresses.sample(10000, replace=True).copy()
addresses["addr_key"] = addresses["addr_key"].astype(str)
addresses

,addr_key,CountryFR,postcode,city,street,housenumber
0,2762939,Belgique,2240,Zandhoven,Kriekenlaan,22
1,2253494,Belgique,8600,Diksmuide,Cardijnlaan(D),*
2,1764140,Belgique,9860,Oosterzele,Poststraat,19
3,87070,Belgique,8300,Knokke-Heist,Sylvain Dupuisstraat,26
4,2261339,Belgique,3630,Maasmechelen,Oude Baan(M),338
...,...,...,...,...,...,...
995,2720764,Belgique,5060,Sambreville,Rue du Wainage(VEL),62
996,386196,Belgique,7520,Tournai,Chaussée de Tournai(R-C),4
997,1474962,Belgique,7331,Saint-Ghislain,Rue Louis Caty(B),32
998,2424889,Belgique,7090,Braine-le-Comte,Drève des Bouleaux,5


In [264]:
addresses = get_addresses("../GISAnalytics/data/geocoding/resto_1000_sample.csv.gz")
# addresses = get_addresses("../GISAnalytics/data/geocoding/best_1000_sample.csv.gz")
addresses["addr_key"] = addresses.index.astype(str)
addresses

,street,housenumber,city,postcode,CountryFR,addr_key
0,Chaussée de Namur,198,LEUZE,5310,Belgique,0
1,Rue de la Halle,11,NAMUR,5000,Belgique,1
2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique,2
3,rue du Centre,200,WAIMES,4950,Belgique,3
4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique,4
...,...,...,...,...,...,...
995,Rue Americaine,90,IXELLES,1050,Belgique,995
996,Kloosterstraat,60,EKEREN,2180,Belgique,996
997,Markt,13,BALEN,2490,Belgique,997
998,Lange Lobroekstraat,65,ANVERS,2060,Belgique,998


In [349]:
addresses = pd.concat([
    get_addresses("../GISAnalytics/data/geocoding/resto_1000_sample.csv.gz"),
    get_addresses("../GISAnalytics/data/geocoding/best_1000_sample.csv.gz"),
    get_addresses("address.csv.gz")])
addresses = addresses.reset_index(drop=True)
addresses["addr_key"] = addresses.index.astype(str)

In [312]:
# addresses = addresses[addresses.addr_key.isin([ "2078829"])]#"1622",
# addresses


### Simple way

In [338]:
addresses_seq = addresses.copy()

t = datetime.now()
addresses_seq["json"] = addresses_seq.fillna("").progress_apply(call_ws, check_result=False, structured_osm=False, axis=1)
tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses_seq.shape[0]/tot_time:.2f} it/s")
# KBO dataset:
# Normal mode: 128.78 seconds, 7.77 it/s
# Fastmode:     68.80 seconds, 14.54 it/s

#Resto dataset: 
# Normal mode: 145.73 seconds, 6.86 it/s
# Fast mode:    82.99 seconds, 12.05 it/s

# Best dataset:
# Normal mode: 108.53 seconds, 9.21 it/s
# Fast mode: 37.44 seconds, 26.71 it/s

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [315]:
expand_json(addresses_seq)
addresses_seq

,addr_key,CountryFR,postcode,city,street,housenumber,json,status,time,timing,method,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city
0,2762939,Belgique,2240,Zandhoven,Kriekenlaan,22,"{'match': [{'addr_out_city': 'Zandhoven', 'add...",match,2.418637,"{'fast': 23.936, 'fast > osm': 18.311, 'global...",fast,Kriekenlaan,22,2240,Zandhoven
1,2253494,Belgique,8600,Diksmuide,Cardijnlaan(D),*,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,1.520868,"{''t&p > transformer > libpostal': 114.104, ''...",nostreet,,,8600,Diksmuide
2,1764140,Belgique,9860,Oosterzele,Poststraat,19,"{'match': [{'addr_out_city': 'Issegem', 'addr_...",match,0.033189,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Poststraat,,9860,Issegem
3,87070,Belgique,8300,Knokke-Heist,Sylvain Dupuisstraat,26,"{'match': [{'addr_out_city': 'Knokke-Heist', '...",match,0.025125,"{'fast': 21.195, 'fast > osm': 16.195999999999...",fast,Sylvain Dupuisstraat,24;26;28,8300,Knokke-Heist
4,2261339,Belgique,3630,Maasmechelen,Oude Baan(M),338,"{'match': [{'addr_out_city': 'Maasmechelen', '...",match,0.092662,"{'fast': 89.84100000000001, 'fast > osm': 84.8...",fast,Oude Baan,,3630,Maasmechelen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2720764,Belgique,5060,Sambreville,Rue du Wainage(VEL),62,"{'match': [{'SIM_street_which': '', 'addr_out_...",match,0.152240,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",regex[init],Rue du Wainage,62,5060,Sambreville
996,386196,Belgique,7520,Tournai,Chaussée de Tournai(R-C),4,"{'match': [{'addr_out_city': 'Ramegnies-Chin',...",match,0.078074,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Chaussée de Tournai,,7520,Ramegnies-Chin
997,1474962,Belgique,7331,Saint-Ghislain,Rue Louis Caty(B),32,"{'match': [{'addr_out_city': 'Saint-Ghislain',...",match,0.048020,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Rue Louis Caty,,7331,Saint-Ghislain
998,2424889,Belgique,7090,Braine-le-Comte,Drève des Bouleaux,5,{'match': [{'addr_out_city': 'Braine-le-Comte'...,match,0.033540,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Drève des Bouleaux,,7090,Braine-le-Comte


In [150]:
addresses_seq.method.value_counts()

fast                             979
libpostal+regex[lpost]+photon     11
nostreet                           7
regex[init]                        2
nonum                              1
Name: method, dtype: int64

### Using Dask

In [229]:
addresses_dask = addresses.copy()

In [372]:
t = datetime.now()
dd_addresses = dd.from_pandas(addresses_dask, npartitions=8)

dask_task = dd_addresses.apply(call_ws, check_result=False, meta=('x', 'str'), axis=1)

with ProgressBar(): 
    addresses_dask["json"] = dask_task.compute()
    
tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses_dask.shape[0]/tot_time:.2f} it/s")
# KBO dataset:
# Normal mode: 24.52 seconds, 40.79 it/s
# Fastmode:    15.81 seconds, 63.27 it/s


# Resto dataset:
# Normal mode: 27.86 seconds, 35.89 it/s
# Fast mode:   18.44 seconds, 54.23 it/s

# Best dataset: 
# Normal mode: 16.11 seconds, 62.07 it/s
# Fast mode:    9.76 seconds, 102.42 it/s

[########################################] | 100% Completed | 17.5s
17.55 seconds, 56.97 it/s


In [90]:
# 1000, 1 worker: 4m18
# 4 workers, npart=4 : 1m20
# 8 workers, npart=4 : 1m20
# 8 workers, npart=8 : 44s

# with checker=False:
# 8 workers, npart=8 : 24s


In [370]:
expand_json(addresses_dask)
addresses_dask

,street,housenumber,city,postcode,CountryFR,addr_key,json,status,time,timing,method,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city
0,Chaussée de Namur,198,LEUZE,5310,Belgique,0,"{'match': [{'addr_out_city': 'Leuze', 'addr_ou...",match,0.030375,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Chaussée de Namur,198,5310,Leuze
1,Rue de la Halle,11,NAMUR,5000,Belgique,1,"{'match': [{'addr_out_city': 'Namur', 'addr_ou...",match,0.032265,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Rue de la Halle,,5000,Namur
2,Avenue Jacques Sermon,38,GANSHOREN,1083,Belgique,2,"{'match': [{'addr_out_city': 'Ganshoren', 'add...",match,0.035799,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Avenue Jacques Sermon - Jacques Sermonlaan,38,1083,Ganshoren
3,rue du Centre,200,WAIMES,4950,Belgique,3,"{'match': [{'addr_out_city': 'Waimes', 'addr_o...",match,0.045275,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Rue du Centre,,4950,Waimes
4,Kleine Waterstraat,9,HULSHOUT,2235,Belgique,4,"{'match': [{'addr_out_city': 'Hulshout', 'addr...",match,0.030662,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Kleine Waterstraat,9,2235,Hulshout
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Rue Americaine,90,IXELLES,1050,Belgique,995,{'match': [{'addr_out_city': 'Ixelles - Elsene...,match,0.029608,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Rue Américaine - Amerikaanse straat,,1050,Ixelles - Elsene
996,Kloosterstraat,60,EKEREN,2180,Belgique,996,"{'match': [{'addr_out_city': 'Ekeren', 'addr_o...",match,0.024305,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Kloosterstraat,60,2180,Ekeren
997,Markt,13,BALEN,2490,Belgique,997,"{'match': [{'addr_out_city': 'Balen', 'addr_ou...",match,0.022344,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Markt,13,2490,Balen
998,Lange Lobroekstraat,65,ANVERS,2060,Belgique,998,"{'match': [{'addr_out_city': 'Antwerpen', 'add...",match,0.022515,"{''t&p > transformer > libpostal': 0.0, ''t&p ...",fast,Lange Lobroekstraat,65,2060,Antwerpen


In [201]:
addresses_dask.method.value_counts()#.json.loc[550]

fast                             979
libpostal+regex[lpost]+photon     11
nostreet                           7
regex[init]                        2
nonum                              1
Name: method, dtype: int64

In [232]:
mg = addresses_seq[["addr_key", "city", "postcode","street", "housenumber", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city"]].merge(
    addresses_dask[["addr_key", "city", "postcode","street", "housenumber", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city"]], how="outer", indicator=True)
if mg.shape[0] == addresses.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

Same result in seq and dask run!


In [233]:
mg[mg._merge != "both"].sort_values("addr_key")


,addr_key,city,postcode,street,housenumber,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city,_merge


## Batch calls (batch WS)

### Single block

In [347]:
t = datetime.now()

addresses_batch = call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], 
                                mode="long", 
                                check_result=False, 
                                structured_osm=False)

tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses.shape[0]/tot_time:.2f} it/s")
# KBO dataset: 33.94 seconds, 29.46 it/s
# Best:        24.99 seconds, 40.01 it/s
# Resto:       38.33 seconds, 26.09 it/s

KeyboardInterrupt: 

In [344]:
addresses

,addr_key,CountryFR,postcode,city,street,housenumber
424,1376463,Belgique,3511,Hasselt,Olmenbosstraat,25
654,2332135,Belgique,9820,Merelbeke,Waterstraat,2
707,1195503,Belgique,2660,Antwerpen,Aartselaarstraat,34
617,2651001,Belgique,1630,Linkebeek,Clos Roger Thiéry,2
257,2392434,Belgique,8850,Ardooie,Stationsstraat,3
...,...,...,...,...,...,...
822,1956739,Belgique,8400,Oostende,Nieuwpoortsesteenweg,604
121,2899607,Belgique,3500,Hasselt,Kempische steenweg,202
151,309015,Belgique,9451,Haaltert,Rijstraat(KE),11
631,2489037,Belgique,8340,Damme,Stationsstraat,97


In [242]:
mg = addresses_seq[[ "city", "postcode","street", "housenumber", "method", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city", "addr_key"]].fillna("").replace("fast", "orig").merge(
    addresses_batch[["city", "postcode","street", "housenumber", "method", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city", "addr_key"]].fillna(""), how="outer", indicator=True)
if mg[mg._merge == "both"].shape[0] == addresses.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

!!! Not the same result in seq and dask run!


In [243]:
mg[mg._merge != "both"]

,city,postcode,street,housenumber,method,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city,addr_key,_merge
46,LIVES-SUR-MEUSE,5101,Chaussée de Marche,563,none,,,,,46,left_only
784,SART-LEZ-SPA,4845,route de Balmoral,35,none,,,,,784,left_only


In [213]:
# Geocode + address
call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], mode="geo", check_result=False) 

,addr_key,lat,lon,place_rank,method
0,0,50.956670,3.641552,26.0,orig
1,1,50.911196,4.715074,30.0,orig
2,2,50.826581,4.302950,30.0,orig
3,3,51.184610,3.567174,26.0,orig
4,4,51.028284,4.603456,30.0,orig
...,...,...,...,...,...
995,233,50.848531,3.537555,14.0,nostreet
996,874,50.848531,3.537555,14.0,nostreet
997,207,51.057013,3.799241,21.0,nostreet
998,460,50.816471,3.339828,21.0,nostreet


In [249]:
# Geocode + address
call_ws_batch(addresses[[addr_key_field, street_field, housenbr_field, postcode_field, city_field, country_field]], mode="short", check_result=False) 

,addr_key,lat,lon,place_rank,method,place_id,addr_out_street,addr_out_number,in_house_nbr,lpost_house_nbr,lpost_unit,addr_out_postcode,addr_out_city,addr_out_country
0,2762939,51.211615,4.649820,30.0,orig,1032308.0,Kriekenlaan,22,22,22,,2240,Zandhoven,België / Belgique / Belgien
1,2253494,51.059548,2.870803,21.0,nostreet,4005124.0,None,None,*,,,8600,Diksmuide,België / Belgique / Belgien
2,1764140,50.924678,3.788056,26.0,orig,436186.0,Poststraat,None,19,19,,9860,Issegem,België / Belgique / Belgien
3,87070,51.351333,3.285724,30.0,orig,3675084.0,Sylvain Dupuisstraat,24;26;28,26,26,,8300,Knokke-Heist,België / Belgique / Belgien
4,2261339,50.977155,5.690444,26.0,orig,379663.0,Oude Baan,None,338,,m 338,3630,Maasmechelen,België / Belgique / Belgien
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2720764,50.462361,4.596458,30.0,regex[init],2657266.0,Rue du Wainage,62,62,62,,5060,Sambreville,België / Belgique / Belgien
996,386196,50.639445,3.349058,26.0,orig,1983533.0,Chaussée de Tournai,None,4,4,,7520,Ramegnies-Chin,België / Belgique / Belgien
997,1474962,50.479737,3.838976,26.0,orig,432987.0,Rue Louis Caty,None,32,32,,7331,Saint-Ghislain,België / Belgique / Belgien
998,2424889,50.606398,4.159495,26.0,orig,379561.0,Drève des Bouleaux,None,5,5,,7090,Braine-le-Comte,België / Belgique / Belgien


In [360]:
# Geocode + address, with rejected addresses
call_ws_batch(addresses, mode="long", with_reject=True)

,error
0,Field 'addr_key' cannot contain duplicated val...


### Batch blocs

In [359]:
# addresses = addresses.sample(10000, replace=True)
# addresses = addresses.reset_index(drop=True)
# addresses["addr_key"]= addresses.index.astype(str)

In [352]:
t = datetime.now()

nb_threads=8

chunks = np.array_split(addresses, nb_threads) # addresses.shape[0]//100)

print(f"{len(chunks)} chunks on {nb_threads} threads")

import multiprocess as mp

p = mp.Pool(nb_threads)

def f(chunk):
    return call_ws_batch(chunk, mode="long", 
                        check_result=False, 
                        structured_osm=False)

with p:
     res= list(tqdm(p.imap(f, chunks), total=len(chunks)))
    
addresses_batch2 = pd.concat(res).reset_index(drop=True)

tot_time = (datetime.now() - t).total_seconds()
print(f"{tot_time:.2f} seconds, {addresses.shape[0]/tot_time:.2f} it/s")
# KBO:    9.28 seconds, 107.72 it/s
# Best:   6.88 seconds, 145.43 it/s
# Resto: 11.79 seconds,  84.85 it/s

20 chunks on 20 threads


  0%|          | 0/20 [00:00<?, ?it/s]

23.39 seconds, 128.27 it/s


In [361]:
# addresses_batch2

In [317]:
mg = addresses_seq[[ "city", "postcode","street", "housenumber", "method", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city", "addr_key"]].fillna("").replace("fast", "orig").merge(
    addresses_batch2[["city", "postcode","street", "housenumber", "method", "addr_out_street", "addr_out_number", "addr_out_postcode", "addr_out_city", "addr_key"]].fillna(""), how="outer", indicator=True)
if mg[mg._merge == "both"].shape[0] == addresses.shape[0]:
    print("Same result in seq and dask run!")
else: 
    print("!!! Not the same result in seq and dask run!")
    

Same result in seq and dask run!


In [318]:
mg[mg._merge != "both"].sort_values("addr_key")

,city,postcode,street,housenumber,method,addr_out_street,addr_out_number,addr_out_postcode,addr_out_city,addr_key,_merge


## Comparing options

In [ ]:
addresses = get_addresses("address.csv.gz")
addresses = addresses[addresses[country_field] == "Belgique"]
addresses = addresses.sample(10000).copy()

In [ ]:
results = {}
it_per_seconds=pd.DataFrame()

for check_label in ["check", "nocheck"]:
    for struct_label in ["struct", "unstruct" ]:
        print(check_label, struct_label)
        start=datetime.now()
        
        results[(check_label, struct_label)] = call_ws_batch_chunks(addresses, 
                                                                    mode="short", 
                                                                    check_result   =  check_label == "check", 
                                                                    structured_osm =  struct_label == "struct")
        
        it_per_seconds.loc[check_label, struct_label] = addresses.shape[0] / (datetime.now()-start).total_seconds()
print("Iterations per seconds:")
it_per_seconds

In [ ]:
print("Match rate")
pd.DataFrame({k1: {k2: results[(k1,k2)].shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

In [ ]:
print("Match rate (without nostreet)")
pd.DataFrame({k1: {k2: results[(k1,k2)].query("method!='nostreet'").shape[0]/addresses.shape[0] for k2 in ["struct", "unstruct"]} 
                  for k1 in  ["check","nocheck"]})

In [ ]:
print("Unmatched addresses")
for k1 in results:
    print(k1)
    nomatch=addresses[~addresses[addr_key_field].isin(results[k1]["addr_key"])]
    display(nomatch)
    print(nomatch[country_field].value_counts())

In [ ]:
vc_values = pd.DataFrame(columns=results.keys(), index=results.keys())

for k1 in results:
    vc_values.loc[k1, k1] = results[k1].shape[0]
    for k2 in results:
        if k1>k2:
            r1=results[k1]
            r2=results[k2]
            mg = r1[["addr_key", "place_id"]].merge(r2[["addr_key", "place_id"]], on="addr_key", how="outer", indicator=True)
 
            vc = mg._merge.value_counts()

            mismatches = mg[mg.place_id_x != mg.place_id_y][["addr_key"]]
            mismatches = mismatches.merge(addresses.rename({addr_key_field:"addr_key"}, axis=1))
            mismatches = mismatches.merge(r1[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches = mismatches.merge(r2[["addr_key", "addr_out_street", "addr_out_number", "extra_house_nbr", "addr_out_postcode", "addr_out_city"]], on="addr_key")
            mismatches.columns = pd.MultiIndex.from_arrays([["Input"]*6 + [f"x:{k1}"]*5 + [f"y:{k2}"]*5, mismatches.columns])

            mismatch_values = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).fillna("") != 
                                          mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).fillna("")).any(axis=1)]
            
            mismatch_values_no_nmbr = mismatches[(mismatches[f"x:{k1}"].rename(lambda x: x.replace("_x", ""), axis=1).drop("addr_out_number", axis=1).fillna("") != 
                                                  mismatches[f"y:{k2}"].rename(lambda x: x.replace("_y", ""), axis=1).drop("addr_out_number", axis=1).fillna("")).any(axis=1)]
            
            
            vc_label = f"{vc['both']} ({mismatches.shape[0]} - {mismatch_values.shape[0]} - {mismatch_values_no_nmbr.shape[0]}) / {vc['left_only']} / {vc['right_only']}"
            vc_values.loc[k1, k2]=vc_label

                
            print(f"{k1} vs {k2}")
            print(vc_label)
            print("-----------------------------")
            
            print(f"Only in {k1}")
            display(r1[r1.addr_key.isin(mg[mg._merge=="left_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print(f"Only in {k2}")
            display(r2[r2.addr_key.isin(mg[mg._merge=="right_only"].addr_key)].merge(addresses.rename({addr_key_field:"addr_key"}, axis=1)))
            
            print("Mismatch on place_id")
            display(mismatches)
            
            print("Mismatch on values")
            
            display(mismatch_values)
            
            print("Mismatch on values (no nbr)")
            display(mismatch_values_no_nmbr)
            
            print("#######################")
            
# display(vc_values.fillna(""))

In [ ]:
print("Common in both (disagree on place_id - disagree on values - disagree on values, ignoring number) / results only for row / results only for columns")
vc_values.fillna("")

# tests

In [100]:
osm_host ="172.27.0.64:8080"
def get_osm(addr, accept_language = ""): #lg = "en,fr,nl"
    params = urllib.parse.urlencode({"q": addr,
                                    "format":"jsonv2",
                                    "accept-language":accept_language,
                                    "addressdetails":"1",
                                    "namedetails" : "1",
                                    "limit": "50"
                                    })
    
    url = "http://%s/search.php?%s"%(osm_host, params)
#     print(f"Call to OSM: {url}")
    try: 
        with urllib.request.urlopen(url) as response:
            res = response.read()
            res = json.loads(res)
#             return res
            return [ {field: item[field] for field in ["place_id", "lat", "lon", "display_name", "address", "namedetails", "place_rank", "category", "type"]} for item in res] 
    except Exception as e:
        raise Exception (f"Cannot get OSM results ({osm_host}): {e}") 

In [322]:
%timeit get_osm("Av. Fonsny 20, 1060 Bruxelles")

10.2 ms ± 100 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [323]:
%timeit call_ws_test({street_field:   "Av. Fonsny", \
         housenbr_field: "20",\
         city_field:     "Saint-Gilles",\
         postcode_field: "1060",\
         country_field:  "Belgium"}, check_result=False, structured_osm=False)
# res

1.09 ms ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
